In [1]:
from MultiLog import MLR,np
from pyspark import SparkContext,SparkConf
from sklearn.base import copy
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
def fit_in(x,mlr,classes):
    for it in x:
        X=it[:-1].reshape(1,it.shape[0]-1)
        y=np.array([it[-1]])
        y=y.astype('int')
        mlr.partial_fit(X,y,classes)
    yield mlr
def update_in(m1,m2):
    new_mod=copy.deepcopy(m1)
    new_mod.coefs+=m2.coefs
    new_mod.intercept+=m2.intercept
    return new_mod

def avg_coefs_in(mlr,numpart):
    mlr.coefs/=numpart
    mlr.intercept/=numpart
    return mlr

In [4]:
conf=SparkConf().setAppName("Multilog-SGD").setMaster('local[4]')
sc=SparkContext(conf=conf)

In [6]:
X=mnist.train.images[:10000,:]
y=mnist.train.labels[:10000]
X=np.hstack((X,y.reshape((y.shape[0],1))))

In [7]:
max_iter=450
mlr=MLR()
n_classes=np.unique(y).shape[0]
prev_coef=np.zeros((n_classes,X.shape[1]-1))
prev_intercept=np.zeros(n_classes)
tol=0.0001

In [8]:
for i in range(max_iter):
    dat=sc.parallelize(X)
    mlr=dat.mapPartitions(lambda x:fit_in(x,mlr,n_classes)).reduce(lambda m1,m2:update_in(m1,m2))
    mlr=avg_coefs_in(mlr,dat.getNumPartitions())
    diff_coef=mlr.coefs-prev_coef
    diff_intercept=mlr.intercept-prev_intercept
    coef_norm=np.linalg.norm(diff_coef,ord='fro')**2
    gradient_sum=np.sqrt(coef_norm+np.sum(diff_intercept**2))
    if gradient_sum<tol:
        break
    prev_coef=mlr.coefs
    prev_intercept=mlr.intercept
    np.random.shuffle(X)
print(mlr.coefs)
print(mlr.intercept)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[-3.25790195  0.52796854  0.99558536 -0.67673199  0.03237485  6.51806171
 -4.5775574   3.44522782 -2.33061756 -0.67640938]


In [9]:
mlr.predict(X[:,:-1])

In [10]:
mlr.predictions

array([8, 8, 5, ..., 3, 4, 6], dtype=int64)

In [11]:
np.mean(X[:,-1]==mlr.predictions)

0.99